In [ ]:
!pip install Bio

from Bio import Entrez, SeqIO
from io import StringIO, BytesIO
import xml.etree.ElementTree as ET
import re
import ast
import json
import time
import os


Entrez.email = 'hari.parthasarathy@berkeley.edu'  # Replace with your email address



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.0 MB/s eta 0:00:00


In [ ]:
def search_sra(query, count):

    handle = Entrez.esearch(db = 'sra', term = query, retmax = count)
    records = Entrez.read(handle)
    handle.close()

    id_list = records['IdList']

    for record_id in id_list:
        summary = Entrez.esummary(db = 'sra', id = record_id)
        read_summary = Entrez.read(summary)
        data = read_summary[0]
        summary.close()

        print('Metadata ', data)
        print('\n')

        print('RecordID: ',record_id)
        print('\n')


In [ ]:
search_sra("PRJDB9292", 1)

Metadata  {'Item': [], 'Id': '19702229', 'ExpXml': '<Summary><Title>Illumina MiSeq paired end sequencing of SAMD00204524</Title><Platform instrument_model="Illumina MiSeq">ILLUMINA</Platform><Statistics total_runs="1" total_spots="370845" total_bases="223248690" total_size="121576221" load_done="true" cluster_name="public"/></Summary><Submitter acc="DRA009581" center_name="KURUME_U" contact_name="Biostatistics Center, Kurume University" lab_name="Biostatistics Center, Kurume University"/><Experiment acc="DRX197985" ver="1" status="public" name="Illumina MiSeq paired end sequencing of SAMD00204524"/><Study acc="DRP008133" name="Gut metagenome analysis, Parkinsons disease and history of appendectomy"/><Organism taxid="408170" ScientificName="human gut metagenome"/><Sample acc="DRS226459" name=""/><Instrument ILLUMINA="Illumina MiSeq"/><Library_descriptor><LIBRARY_NAME>DN20</LIBRARY_NAME><LIBRARY_STRATEGY>AMPLICON</LIBRARY_STRATEGY><LIBRARY_SOURCE>METAGENOMIC</LIBRARY_SOURCE><LIBRARY_SELE

In [ ]:
def request_gene_id(query, count, sort_method="relevance"):
  handle = Entrez.esearch(db="gene", term=query, retmax=count, retmode="xml", sort=sort_method)
  record = Entrez.read(handle)
  return record['IdList']

In [ ]:
id_list = request_gene_id("Hemoglobin", 2)
print(id_list)

['820216', '816103']


In [ ]:
def collate_ids(id_list):
  ret = id_list[0]
  for id in id_list[1:]:
    ret += ","
    ret += id
  return ret

In [ ]:
id_string = collate_ids(id_list)
print(id_string)

820216,816103


In [ ]:
def fetch_records(ids):
  handle = Entrez.efetch(db="gene", id=ids, retmode="xml")
  records = Entrez.read(handle)
  return list(records)

In [ ]:
records = fetch_records(id_string)

In [ ]:
def get_nuc_id_and_range(record):
  seq_select = []
  try:
    seq_select = list(filter(lambda x: x["Gene-commentary_label"]=="RefSeqGene", record['Entrezgene_locus']))[0]
  except:
    seq_select = record['Entrezgene_locus'][0]
    print(f'Gene {record["Entrezgene_track-info"]["Gene-track"]["Gene-track_geneid"]} does not have an independent RefSeq, defaulting to first entry')
  try:
    seq_interval = seq_select["Gene-commentary_seqs"][0]["Seq-loc_int"]["Seq-interval"]
    return (seq_interval["Seq-interval_id"]["Seq-id"]["Seq-id_gi"], seq_interval["Seq-interval_from"],  seq_interval["Seq-interval_to"])
  except:
    print("Nucleotide sequence reference not found")
    return ("SKIP", "SKIP", "SKIP")

In [ ]:
info_list = [get_nuc_id_and_range(records[i]) for i in range(len(records))]
print(info_list)

KeyError: 'Entrezgene_locus'

In [ ]:
def fetch_nucleotide_records(ids):
  handle = Entrez.efetch(db="nuccore", id=ids, rettype="fasta", retmode="xml")
  records = Entrez.read(handle)
  return list(records)

In [ ]:
id_list, start_list, end_list = zip(*info_list)
records = fetch_nucleotide_records(collate_ids(id_list))

In [ ]:
print(start_list)
print(end_list)
print(records[0]["TSeq_sequence"][int(start_list[0]):int(end_list[0])])

('3276162', '6982530')
('3277929', '6983776')
GTTTTAGTGTACTGTTTTTTAAAAAAAGAAAAAATTGAACTAAAATAGATCCAAAATCTTGTGTCTGGAGAAGTAAAATTATTTGTAGAATGATTGTAAAATGATCCCCACGTTAACATCCATCTTGTGAAAATAAATACTTCAAACCATGATATTTCTCACGAATAACAATTTCAAGAAGAACTACAGTATCACGTATCTCCCTAATGATAGCACAAGATCCTTCTTTATAATTAAAATAGTGAACTGTAGAAAGTCAGAGAGCAACATAGTACACACACAGAAACACGTATATGTGTATCATGTATGTGGAATAGATTCATGTATGCGATACCCAAATGATCAATAGGGTTTTATGACTCTTCTTGTTTCATCTCGGTCTTGATGGCTAAAGCCAAGTGATCATAAGCTTGAGACCAAGCACCTTCCACTTCTTCATTGTATTTCTCCCCCAACCCCTCTTTCAATGTCCTTAGCAAAGCTTCTTTCACCACCTGCTTTCACATTCACACCTTATTAAAATGCGTTCAAATCGAAAATCAAAATCAGCCAAAATTAATGTGTATATATATTTTTTTTTAACATAACAGACCTCGAAGTGAGGGTCAATAACGCCGCTTTTGAGATGAATTGAGCCTAAATATTGGAGGGTTGTGTCAGCCACTACCACCTTTCCTTCCTCCCTCAGCTGTATAGCTGTTTCACATGTCTACACACAAACAAAATAAAATAAAAATTCATACATGTTATGCAAAGTCTTTTTCTTTTTTGTCTGTCTAAAACATGTTATGCAAAGTCTTAGAAGCCAAAAGTGAGTTGAACCAAACCAAGTCATAAACCCAAATTTAAAAAGTAAACGGAATTTGGTTTATATGGTTCTATTATAATAAGATTAGCAAAAAAGTAAAAATCCATAAAGCGACCCGAACCACTTGGATTCTTCAGTAACAATTC

In [ ]:
def get_seqs(info_list, gene_id_list):
  ret = []
  id_list, start_list, end_list = zip(*info_list)
  fil_id_list = list(filter(lambda x: x != "SKIP", id_list))
  records = fetch_nucleotide_records(collate_ids(fil_id_list))

  offset = 0
  for i in range(len(start_list)):
    if id_list[i] == "SKIP":
      ret.append((gene_id_list[i], "NO SEQUENCE FOUND"))
      offset += 1
      continue
    start = int(start_list[i])
    end = int(end_list[i])
    ret.append((gene_id_list[i], records[i - offset]["TSeq_sequence"][start:end]))
  return ret

In [ ]:
get_seqs(info_list)

[('240255695',
  'GTTTTAGTGTACTGTTTTTTAAAAAAAGAAAAAATTGAACTAAAATAGATCCAAAATCTTGTGTCTGGAGAAGTAAAATTATTTGTAGAATGATTGTAAAATGATCCCCACGTTAACATCCATCTTGTGAAAATAAATACTTCAAACCATGATATTTCTCACGAATAACAATTTCAAGAAGAACTACAGTATCACGTATCTCCCTAATGATAGCACAAGATCCTTCTTTATAATTAAAATAGTGAACTGTAGAAAGTCAGAGAGCAACATAGTACACACACAGAAACACGTATATGTGTATCATGTATGTGGAATAGATTCATGTATGCGATACCCAAATGATCAATAGGGTTTTATGACTCTTCTTGTTTCATCTCGGTCTTGATGGCTAAAGCCAAGTGATCATAAGCTTGAGACCAAGCACCTTCCACTTCTTCATTGTATTTCTCCCCCAACCCCTCTTTCAATGTCCTTAGCAAAGCTTCTTTCACCACCTGCTTTCACATTCACACCTTATTAAAATGCGTTCAAATCGAAAATCAAAATCAGCCAAAATTAATGTGTATATATATTTTTTTTTAACATAACAGACCTCGAAGTGAGGGTCAATAACGCCGCTTTTGAGATGAATTGAGCCTAAATATTGGAGGGTTGTGTCAGCCACTACCACCTTTCCTTCCTCCCTCAGCTGTATAGCTGTTTCACATGTCTACACACAAACAAAATAAAATAAAAATTCATACATGTTATGCAAAGTCTTTTTCTTTTTTGTCTGTCTAAAACATGTTATGCAAAGTCTTAGAAGCCAAAAGTGAGTTGAACCAAACCAAGTCATAAACCCAAATTTAAAAAGTAAACGGAATTTGGTTTATATGGTTCTATTATAATAAGATTAGCAAAAAAGTAAAAATCCATAAAGCGACCCGAACCACTTGGATTCTTCAGTAACAATTCTTTATTGACAGAGAAATCTATGTTTGTT

In [ ]:
def get_gene_names(record):
  try:
    return record["Entrezgene_gene"]["Gene-ref"]["Gene-ref_desc"]
  except:
    print(f'Gene {record["Entrezgene_track-info"]["Gene-track"]["Gene-track_geneid"]}: Gene Reference Description not found')

In [ ]:
def get_sequence_from_query(query, count, fout=None):
  gene_id_list = request_gene_id(query, count)
  print(gene_id_list)
  gene_records = fetch_records(collate_ids(gene_id_list))
  info_list = [get_nuc_id_and_range(gene_records[i]) for i in range(len(gene_records))]
  name_list = [get_gene_names(gene_records[i]) for i in range(len(gene_records))]
  return list(zip(name_list, get_seqs(info_list, gene_id_list)))


In [ ]:
try:
  print(get_sequence_from_query("Hemoglobin subunit beta", 20))
except Exception as error:
  print(f"Error Code: {error}")

['3043', '24440', '3047', '3048', '3039', '3040', '3045', '396485', '3044', '100503605', '3046', '15129', '293267', '15130', '450978', '30216', '53335', '419079', '100134871', '476825']
Gene 24440 does not have an independent RefSeq, defaulting to first entry
Gene 396485 does not have an independent RefSeq, defaulting to first entry
Gene 100503605 does not have an independent RefSeq, defaulting to first entry
Gene 15129 does not have an independent RefSeq, defaulting to first entry
Gene 293267 does not have an independent RefSeq, defaulting to first entry
Gene 15130 does not have an independent RefSeq, defaulting to first entry
Gene 450978 does not have an independent RefSeq, defaulting to first entry
Gene 30216 does not have an independent RefSeq, defaulting to first entry
Gene 419079 does not have an independent RefSeq, defaulting to first entry
Gene 100134871 does not have an independent RefSeq, defaulting to first entry
Gene 476825 does not have an independent RefSeq, defaulting to

In [ ]:
get_sequence_from_query("Hemoglobin subunit beta", 3)

['3043', '24440', '3047']
Gene 24440 does not have an independent RefSeq, defaulting to first entry


[('hemoglobin subunit beta',
  ('3043',
   'ACATTTGCTTCTGACACAACTGTGTTCACTAGCAACCTCAAACAGACACCATGGTGCATCTGACTCCTGAGGAGAAGTCTGCCGTTACTGCCCTGTGGGGCAAGGTGAACGTGGATGAAGTTGGTGGTGAGGCCCTGGGCAGGTTGGTATCAAGGTTACAAGACAGGTTTAAGGAGACCAATAGAAACTGGGCATGTGGAGACAGAGAAGACTCTTGGGTTTCTGATAGGCACTGACTCTCTCTGCCTATTGGTCTATTTTCCCACCCTTAGGCTGCTGGTGGTCTACCCTTGGACCCAGAGGTTCTTTGAGTCCTTTGGGGATCTGTCCACTCCTGATGCTGTTATGGGCAACCCTAAGGTGAAGGCTCATGGCAAGAAAGTGCTCGGTGCCTTTAGTGATGGCCTGGCTCACCTGGACAACCTCAAGGGCACCTTTGCCACACTGAGTGAGCTGCACTGTGACAAGCTGCACGTGGATCCTGAGAACTTCAGGGTGAGTCTATGGGACGCTTGATGTTTTCTTTCCCCTTCTTTTCTATGGTTAAGTTCATGTCATAGGAAGGGGATAAGTAACAGGGTACAGTTTAGAATGGGAAACAGACGAATGATTGCATCAGTGTGGAAGTCTCAGGATCGTTTTAGTTTCTTTTATTTGCTGTTCATAACAATTGTTTTCTTTTGTTTAATTCTTGCTTTCTTTTTTTTTCTTCTCCGCAATTTTTACTATTATACTTAATGCCTTAACATTGTGTATAACAAAAGGAAATATCTCTGAGATACATTAAGTAACTTAAAAAAAAACTTTACACAGTCTGCCTAGTACATTACTATTTGGAATATATGTGTGCTTATTTGCATATTCATAATCTCCCTACTTTATTTTCTTTTATTTTTAATTGATACATAATCATTATACATATTTATGGGTTAAAGTGTAATGTTTTAATATGTGTA